In [1]:
import pandas as pd
import math
import sys

# Change path to local path to github repository
local_path = '/Users/omaromeir/Documents/GitHub'
sys.path.insert(0, local_path + '/impact_and_comparative_summarization')

import src.asp
import src.cps

In [2]:
# Postgresql database connection
import psycopg2  
conn = psycopg2.connect(dbname="testdb", host="localhost", port="5432")
conn.set_client_encoding('UTF8')

In [3]:
# SQL query to join movie and genre tables and return all Action movies
query = """SELECT * FROM movie m, genre g WHERE m.id = g.movie_id AND g.genre = 'Action';"""

In [4]:
# Call to impact summary algorithm to summarize the results of the above query
# Number of rules k is set to 4
# The aggregate function is set to 'count'. 'avg', 'sum', 'max' and 'min' are other options
# 'revenue' is the measure attribute, used for all other functions but count
# query and conn are the query and db connection string
# A list of categorical attributes to use for summary rules ['original_language','release_date','production_country','vote_average']
# Maximum weight for rules is set to 3
# The size of sample is set to 300, -1 to get the full set
imp_rules = src.asp.pastWatch(4,'count', 'revenue', query, conn, ['original_language','release_date','production_country','vote_average'], 3, 300)

/Users/omaromeir/Documents/GitHub/impact_and_comparative_summarization/src/asp.py:212: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [5]:
# Print the generated rules
for r in imp_rules[1]:
    r.printRule()

rule: original_language production_country var: en United States of America size:2 score:338 impact:169 abs_score:28730 marginal_coverage:169 abs_coverage:169
rule: original_language var: en size:1 score:73 impact:242 abs_score:242 marginal_coverage:73 abs_coverage:242
rule: original_language production_country vote_average var: en United States of America 6.0 size:3 score:67 impact:67 abs_score:201 marginal_coverage:0 abs_coverage:67
rule: original_language vote_average var: en 5.0 size:2 score:168 impact:84 abs_score:168 marginal_coverage:84 abs_coverage:84


In [6]:
# Comparative summary takes 2 queries, then gets the results of both as two sets of tuples to compare
query1 = """SELECT * FROM movie m, genre g WHERE m.id = g.movie_id AND g.genre = 'Action';"""
query2 = """SELECT * FROM movie m, genre g WHERE m.id = g.movie_id AND g.genre = 'Adventure';"""

In [7]:
# Call to comparative summary algorithm to summarize the commonalities between the above queries
# Number of rules k is set to 4
# The aggregate function is set to 'count'. 'avg', 'sum', 'max' and 'min' are other options
# 'revenue' is the measure attribute, used for all other functions but count
# query1, query2 and conn are the queries and db connection string
# A list of categorical attributes to use for summary rules ['original_language', 'production_country','vote_average']
# Maximum weight for rules is set to 3
# The size of sample is set to 100, -1 to get the full sets
cps_rules = src.cps.pastWatch(4, 'count', 'revenue', query1, query2, ['original_language','production_country','vote_average'], conn, 100)

/Users/omaromeir/Documents/GitHub/impact_and_comparative_summarization/src/cps.py:514: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql_query(query1, conn)
/Users/omaromeir/Documents/GitHub/impact_and_comparative_summarization/src/cps.py:521: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql_query(query2, conn)


In [8]:
# Print the generated rules
for r in cps_rules[2]:
    r.printRule()

rule: original_language var: en size:1 score:12798 impact:12798 abs_score:0 marginal_coverage:6399 abs_coverage:6399
rule: vote_average var: 5.0 size:1 score:682 impact:2016 abs_score:0 marginal_coverage:341 abs_coverage:1008
rule: original_language production_country var: en United States of America size:2 score:10224 impact:5616 abs_score:0 marginal_coverage:2556 abs_coverage:2808
rule: production_country var: United States of America size:1 score:5112 impact:5616 abs_score:0 marginal_coverage:2556 abs_coverage:2808
